In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model


In [2]:
s=pd.read_csv(r'C:\Users\Lenovo\Downloads\Sentiment.csv',header=0)

In [3]:
s.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
s.shape

(13871, 21)

In [5]:
s=s[['text','sentiment']]

In [6]:
s

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [7]:
s=s[s['sentiment']!='Neutral']

In [8]:
s=pd.DataFrame(s)

In [9]:
s['sentiment'].value_counts()

Negative    8493
Positive    2236
Name: sentiment, dtype: int64

SO THE TOTAL NEGATIVE SENTIMENTS ARE 8493 AND POSITIVE SENTIMENTS ARE 2236 PRESENT IN THE DATA

In [10]:
s

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
english_stops = set(stopwords.words('english'))

In [13]:
s

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [14]:
text_data = s['text']
# Concatenate all the text into a single string
combined_text = ''.join(text_data)

# Get unique characters using the set() function
unique_chars = set(combined_text)

# Print the unique characters
print(unique_chars)

{'😋', 'S', '―', '↓', '4', '🏻', 'K', '🏾', 't', 'T', '`', '👉', '—', '🐍', '⁰', '…', '🔪', '·', '🎪', '😳', '0', ')', '😁', 'U', '☞', '\U000fe320', '~', '🍺', 'c', '👎', '8', '3', '🏼', '\xa0', 'R', '✋', '😭', '🎉', '🐘', '😒', "'", '6', 'x', '“', '》', '🇸', '🔇', '‼', '\U000fe327', '^', 'D', 'u', ';', '💪', 'y', '✯', 'e', 'O', '😏', 'H', 'l', '🚀', 'j', '/', '😞', '📢', 'k', '$', '=', 'F', '❤', '😡', '.', ']', '\U000fe4ec', '→', '✔', 'î', ',', 'm', '|', '👊', 'f', '🇺', '・', '🙊', ' ', 'é', '►', 'G', '😠', 'v', 's', 'p', '?', '😴', '-', '@', 'Y', '🙈', '😜', '👋', '+', '‘', 'N', '–', '👍', 'P', '😊', '♀', '⇒', '7', '(', 'A', 'V', 'a', '↑', '😕', '’', '✅', '😑', 'h', 'E', 'M', '5', ':', '😐', 'C', '9', 'o', '💨', '!', '👏', '_', 'q', '*', '&', 'b', '💀', 'w', 'z', 'B', '💩', '😩', 'd', '💁', '2', '😲', '☜', '%', '😉', '😄', '◄', '😆', '😀', '😎', '👭', 'I', '�', '1', '➡', '🙌', '👌', 'Q', '\n', '🍷', '😍', '☑', '😂', '👀', '💛', 'i', '»', '⚡', '\U000fe4e6', 'r', '☢', 'W', 'à', '[', '🔥', '😻', '"', '🏿', 'X', '#', 'g', '️', 'ł', '”', '🌷', '💧',

In [15]:
 # PRE-PROCESS REVIEW
s = s.replace({'<.*?|=@\U000fe4e6✔%$\U000fe4ec\U000fe327!—»>': ''}, regex = True)          # remove html tag
s=s.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
s['text']= s['text'].apply(lambda x: [w for w in x.split() if w not in english_stops])  # remove stop words
s['text']= s['text'].apply(lambda x: [w.lower() for w in x])   # lower case
    

SO THERE ARE LOTS OF UNIQUE CHARACTER PRESENT AS YOU CAN SEE BUT I TREID COVERING MANY OF THEM TO CREATE A BETTER MODEL 

In [16]:
s

,text,sentiment
1,"[rt, scottwalker, didn, catch, full, gopdebate...",Positive
3,"[rt, robgeorge, that, carly, fiorina, trending...",Positive
4,"[rt, danscavino, gopdebate, w, realdonaldtrump...",Positive
5,"[rt, gregabbott, tx, tedcruz, on, first, day, ...",Positive
6,"[rt, warriorwoman, i, liked, happy, i, heard, ...",Negative
...,...,...
13866,"[rt, cappy, yarbrough, love, see, men, never, ...",Negative
13867,"[rt, georgehenryw, who, thought, huckabee, exc...",Positive
13868,"[rt, lrihendry, tedcruz, as, president, i, alw...",Positive
13869,"[rt, jrehling, gopdebate, donald, trump, says,...",Negative


In [17]:
x_data = s['text']       # Reviews/Input
y_data = s['sentiment']    # Sentiment/Output

In [18]:
# ENCODE SENTIMENT -> 0 & 1
y_data = y_data.replace('Positive', 1)
y_data = y_data.replace('Negative', 0)

In [19]:
y_data 

1        1
3        1
4        1
5        1
6        0
        ..
13866    0
13867    1
13868    1
13869    0
13870    1
Name: sentiment, Length: 10729, dtype: int64

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
4300     [ted, cruz, embarrassingly, simplistic, kill, ...
11965    [rt, rwsurfergirl, why, chris, wallace, ask, p...
273      [rt, lesleyabravanel, trump, sycophants, want,...
10771    [rt, laurenreeves, stay, tuned, closing, state...
12997    [how, many, times, trump, declared, bankruptcy...
                               ...                        
12397    [rt, goldietaylor, huckabee, gopdebates, http,...
13287    [rt, rwsurfergirl, jeb, bush, reminds, elevato...
2949     [davids, brain, all, talk, abt, pro, life, amp...
10823    [rt, monaeltahawy, can, someone, send, gopdeba...
10461    [is, ptl, club, where, tammy, mascara, gopdeba...
Name: text, Length: 8583, dtype: object 

6860     [i, disappointed, megynkelly, never, bothered,...
3296     [does, trump, negotiate, price, voter, gopdeba...
7335     [megynkelly, really, awful, last, night, hope,...
11801       [forget, gopdebates, let, talk, ronald, raven]
4822     [the, nsa, spat, randpaul, chrischristie, indi...
    

In [21]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [22]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  138    33  2307 ...  2308  3618  2795]
 [    2     7    64 ...     1     0     0]
 [    2  2796     8 ...    16   241  1972]
 ...
 [ 5478   491   207 ...  5190   825   843]
 [    2   109   199 ...  1186     0     0]
 [   51 13384 13385 ...     0     0     0]] 

Encoded X Test
 [[   4  563   20 ...    5    0    0]
 [ 810    8 2999 ...  388 3132  411]
 [  20   34  949 ... 1079 1339    1]
 ...
 [   2 1578  283 ...   14  509    1]
 [1124    9  212 ...    0    0    0]
 [  85  126 1125 ...    5   60    0]] 

Maximum review length:  13


In [23]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping

# Define the RNN model
model = Sequential()

# Add an embedding layer with 32 dimensions
model.add(Embedding(total_words,output_dim=32, input_length = max_length))

# Add a SimpleRNN layer with 128 units
model.add(SimpleRNN(128))

# Add a fully connected layer with 1 unit (for binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# To prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test),callbacks=[early_stopping])

Epoch 1/5
269/269 [==============================] - 3s 8ms/step - loss: 0.4374 - accuracy: 0.8158 - val_loss: 0.3703 - val_accuracy: 0.8467
Epoch 2/5
269/269 [==============================] - 2s 7ms/step - loss: 0.2319 - accuracy: 0.9068 - val_loss: 0.4025 - val_accuracy: 0.8355
Epoch 3/5
269/269 [==============================] - 2s 7ms/step - loss: 0.1235 - accuracy: 0.9558 - val_loss: 0.5220 - val_accuracy: 0.8099
Epoch 4/5
269/269 [==============================] - 2s 7ms/step - loss: 0.0962 - accuracy: 0.9669 - val_loss: 0.4590 - val_accuracy: 0.8299


In [24]:
y_pred = model.predict((x_test)>0.5).astype('int32')

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

68/68 [==============================] - 0s 2ms/step
Correct Prediction: 1704
Wrong Prediction: 442
Accuracy: 79.40354147250699


SO WE CAN MAKE A CONCLUSION FROM THE REPORT THAT THE MODEL IS GIVING Correct Prediction=1704 AND Wrong Prediction=442 WITH 
Accuracy: 79.40.SO OVERALL IT IS A GOOD MODEL. 

# Based on the model, check the sentiment for the following two sentences
# a.'He is a great leader '
# b.'He is a terrible leader'


In [25]:
review = str(input('Sentiment Review: '))

Sentiment Review: 'He is a great leader '


In [26]:
import re
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  He is a great leader 
Filtered:  ['he great leader ']


In [27]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[ 81  80 266   0   0   0   0   0   0   0   0   0   0]]


In [28]:
result =model.predict(tokenize_words)
print(result)

1/1 [==============================] - 0s 108ms/step
[[0.81258446]]


In [29]:
if result >= 0.7:
    print('Positive')
else:
    print('Negative')

Positive


In [30]:
review = str(input('Sentiment Review: '))

Sentiment Review: 'He is a terrible leader'


In [31]:
import re
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  He is a terrible leader
Filtered:  ['he terrible leader']


In [32]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[ 81 870 266   0   0   0   0   0   0   0   0   0   0]]


In [33]:
result =model.predict(tokenize_words)
print(result)

1/1 [==============================] - 0s 16ms/step
[[0.26398912]]


In [34]:
if result >= 0.7:
    print('Positive')
else:
    print('Negative')

Negative
